In [ ]:
# Import collection metadata
import cPickle as pickle


In [ ]:
# First take a peek at the data
sentence_list = pickle.load(open('data-txt-insensitive/nature05699.p'))

In [ ]:
len(sentence_list)
sentence_list[0]

In [ ]:
from os import listdir
from os.path import isfile, join
import cPickle as pickle

# Define an iterator for streaming the pickle files
class DirectorySentenceIterator:
    def __init__(self, start, at_a_time, filedir):
        self.batch_index = start # global index of start of batch
        self.at_a_time = at_a_time
        
        # Get list of files in directory
        self.file_list = [f for f in listdir(filedir) if isfile(join(filedir, f))]

        self.batch = []
        self.filedir = filedir
        self.index = 0 # within the batch
        self.last = False
        self._update_batch()

    # Makes use of the current index to update the batch
    def _update_batch(self):

        self.batch = []
        
        end_ind = self.batch_index + self.at_a_time
        if end_ind >= len(self.file_list):
            end_ind = len(self.file_list)
            self.last = True
        batch_files = self.file_list[self.batch_index:end_ind]
        
        for fname in batch_files:
            sentences = pickle.load(open(join(self.filedir, fname), 'rb'))
            self.batch.extend(sentences)
            
        if len(self.batch) == 0:
            if not self.last:
                self.batch_index += self.at_a_time
                self._update_batch()
            else:
                raise StopIteration
        
    def __iter__(self):
        return self

    def next(self):
        if self.index >= len(self.batch):
            if self.last:
                raise StopIteration
            else:
                self.index = 0
                self.batch_index += self.at_a_time
                self._update_batch()
        self.index += 1
        return self.batch[self.index - 1]

In [ ]:
dsi = DirectorySentenceIterator(0, 10000, './data-txt-insensitive')
hack = []
for sentence_list in dsi:
    hack.append(sentence_list)

In [5]:
import logging, gensim
from gensim.models import word2vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
model = word2vec.Word2Vec(hack, size=50, window=10, min_count=5)

In [8]:
#model.save('word2vec_txt_insensitive.m')
model = word2vec.Word2Vec.load('word2vec_txt_insensitive.m')

2018-02-28 01:52:57,793 : INFO : loading Word2Vec object from word2vec_txt_insensitive.m
2018-02-28 01:52:57,998 : INFO : loading wv recursively from word2vec_txt_insensitive.m.wv.* with mmap=None
2018-02-28 01:52:58,002 : INFO : setting ignored attribute syn0norm to None
2018-02-28 01:52:58,003 : INFO : setting ignored attribute cum_table to None
2018-02-28 01:52:58,004 : INFO : loaded word2vec_txt_insensitive.m


In [9]:
model.wv['rain']
#model.wv.most_similar('ocean')
model.wv.most_similar('china')

2018-02-28 01:53:03,074 : INFO : precomputing L2-norms of word weight vectors


[(u'india', 0.7654091119766235),
 (u'nigeria', 0.7598310708999634),
 (u'India', 0.7591868042945862),
 (u'Pakistan', 0.7487422823905945),
 (u'brazil', 0.741621732711792),
 (u'pakistan', 0.739411473274231),
 (u'Indonesia', 0.7333608269691467),
 (u'Cuba', 0.7291191816329956),
 (u'indonesia', 0.7280763387680054),
 (u'bangladesh', 0.7261566519737244)]

In [10]:
vocab = list(model.wv.vocab)
X = model[vocab]

In [12]:
from sklearn.manifold import TSNE

In [15]:

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

In [13]:
import pickle
pickle.dump(tsne, open('X_tsne_simple.p', 'wb'))
#X_tsne = pickle.load(open('X_tsne_simple.p', 'rb'))

In [14]:
import pandas as pd
df = pd.DataFrame(X_tsne, index=vocab, columns=['x', 'y'])
#df.to_csv('X_tsne.csv')

ValueError: Shape of passed values is (2, 27339), indices imply (2, 39684)

In [ ]:
import matplotlib.patheffects as path_effects
reload(path_effects)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 6), dpi=150)
ax = fig.add_subplot(1, 1, 1)

ax.scatter(df['x'], df['y'], c='cyan', alpha=0.25)
ax.set_facecolor('black')

#idx = 6
idx = 2

for word, pos in df.iterrows():
    #idx += 1
    idx += 1
    if idx % 160 != 0:
        continue
    
    ax.annotate(word, pos, color='white', fontweight='bold')

plt.show()



In [ ]:
fig.savefig('MINE')

Time for doc2vec

In [ ]:
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

# We need to feed it labeled sentences
idx = 0
doc_sentences = []
for sentence_list in sentences:
    
    ln = metadata[idx]['Collection']['LongName']
    
    for sentence in sentence_list:
        ls = TaggedDocument(words=filter(None, sentence.split(' ')), tags=[unicode(idx), ln])
        doc_sentences.append(ls)
        
    idx += 1



In [ ]:
doc_sentences[0]

In [ ]:
model = doc2vec.Doc2Vec(doc_sentences, size=100, window=8, min_count=5, workers=7)

In [ ]:
model.save('doc2vec_structured.m')

In [ ]:
# Now lets see which is the most similiar to a chosen document

model.docvecs.most_similar(200) 

In [ ]:
print metadata[200]['Collection']['ShortName']

This works, but what would have happened if we had used a different representation of documents that captured less of the structure?

In [ ]:
documents[0]

In [ ]:
import multiprocessing

# Parallelize for speed
pool = multiprocessing.Pool()
documents = pool.map(struct2Doc, metadata)

In [ ]:
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

# We need to feed it labeled sentences
idx = 0
doc_sentences2 = []
for document in documents:
    ln = metadata[idx]['Collection']['LongName']
    sentence = filter(None, document.split(' '))
    td = TaggedDocument(words=sentence, tags=[unicode(idx), ln])
    doc_sentences2.append(td)
    idx += 1

In [ ]:
model_simplified = doc2vec.Doc2Vec(doc_sentences2, size=100, window=8, min_count=5, workers=7)

In [ ]:
model_simplified.save('doc2vec_simple.m')

Let's compare how this version of documents compares

In [ ]:
model_simplified.docvecs.most_similar(200) 

In [ ]:
model.docvecs.most_similar(200)

Looks about the same.

Now let's use LDA on the data. First we have to make a corpus.

In [ ]:
from gensim import corpora

plain_sentences = [filter(None, document.split(' ')) for document in documents]
dictionary = corpora.Dictionary(plain_sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in plain_sentences]

In [ ]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=100, id2word=dictionary)

In [ ]:
lda.print_topics(10)

In [ ]:
lda.save('lda_simple.m')